In [17]:
import pandas as pd

In [18]:
filename = 'dataset1m.xlsx'# <- filename
df = pd.read_excel(filename)

In [33]:
# Flags
ANON_JOBS = True
ANON_ADDRESS = True
ANON_SALARY = True
LOCAL_SUPRESS = True

if LOCAL_SUPRESS:
    REQUIRED_K_ANON = 5

SAVE_ANON_FILE = False

In [20]:
def salary_local_generalization(x):
    val = int(x)
    if val > 100000:
        return "Very high"
    elif val > 75000:
        return "High"
    elif val > 55000:
        return "Mid"
    else:
        return "Low"
def del_col(x):
    return '***'
def addr_mask(x):
    return str(x.split(" д. ")[0]) + ' ***'
if ANON_SALARY:
    df["Заработная плата"] = df["Заработная плата"].apply(salary_local_generalization)

# Прямой идентефикатор - всегда анонимизируем
df["ФИО работника"] = df["ФИО работника"].apply(del_col)


if ANON_ADDRESS:
    df["Адрес работы"] = df["Адрес работы"].apply(addr_mask)

In [21]:
def job_local_generalization(x):
    s = str(x)
    if x == "Административный директор" or x == "Директор по маркетингу" or x == "Финансовый директор" or x == "Генеральный директор":
        return "Директор"
    elif x == "Водитель" or x == "Комендант" or x == "Охранник" or x == "Уборщик" or x == "Секретарь" or x == "Бухгалтер":
        return "Вспомогательный персонал"
    else:
        return s

if ANON_JOBS:
    df["Должность"] = df["Должность"].apply(job_local_generalization)

In [22]:
def phone_local_generalization(x):
    s = str(x)
    if s == "*":
        return s
    elif int(s[1:4]) in [929, 921, 931]:
        return "Megafon"
    elif int(s[1:4]) in [911, 981]:
        return "MTS"
    elif int(s[1:4]) in [961, 962, 963, 964, 903, 905, 906, 909, 960]:
        return "Beeline"
    elif int(s[1:4]) in [901, 952, 904, 950, 951]:
        return "Tele2"

# Прямой идентефикатор - всега анонимизируем
df["Номер телефона"] = df["Номер телефона"].apply(phone_local_generalization)

In [23]:
def get_dict_and_k_anonimity():
    mydictionary = {}
    def getstrline(ind):
        return str(df["ФИО работника"].iloc[ind]) + str(df["Номер телефона"].iloc[ind]) + str(df["Адрес работы"].iloc[ind]) + str(df["Должность"].iloc[ind]) + str(df["Заработная плата"].iloc[ind])
    for row in range(df.shape[0]):
        s = getstrline(row)
        if s in mydictionary:
            mydictionary[s] += 1
        else:
            mydictionary[s] = 1
    return mydictionary, min(mydictionary.values())

In [24]:
print("BEFORE LOCAL SUPPRESSION")
mydict, k_anon = get_dict_and_k_anonimity()
print("K-anon = ", k_anon)

BEFORE LOCAL SUPPRESSION
K-anon =  2


In [25]:
sorted_mydict = sorted(mydict.items(), key=lambda x:x[1])

print("Unique rows:", len(mydict))
print("5 least frequent rows:")
for i in range(5):
    print(sorted_mydict[i])

Unique rows: 4803
5 least frequent rows:
('***Tele2Обводного канала наб. ***ПрограммистHigh', 2)
('***Tele2Обводного канала наб. ***ПрограммистVery high', 4)
('***Beelineпр-кт Малый В.О. ***ПрограммистVery high', 5)
('***MTSпр-кт Малый В.О. ***ПрограммистVery high', 5)
('***BeelineОбводного канала наб. ***ПрограммистVery high', 5)


In [26]:
df

,ФИО работника,Номер телефона,Адрес работы,Должность,Заработная плата
0,***,Megafon,ул. Чайковского ***,Директор,High
1,***,Beeline,пр-кт Владимирский ***,Директор,High
2,***,MTS,ул. Чайковского ***,Директор,High
3,***,MTS,Павлоградский пер. ***,Директор,High
4,***,Tele2,ул. Марата ***,Директор,High
...,...,...,...,...,...
997594,***,Beeline,пр-кт Литейный ***,Директор,High
997595,***,Beeline,ул. Захарьевская ***,Директор,High
997596,***,Megafon,Свечной пер. ***,Менеджер,Low
997597,***,MTS,пр-кт Средний В.О. ***,Менеджер,Low


In [27]:
r = [''.join(val) for val in df.astype(str).values.tolist()]

index = 0
forDelete = []

for index in range(df.shape[0]):
    line = r[index]
    if mydict[line] < REQUIRED_K_ANON:
        #print(line, ">>", index)
        forDelete.append(index)
    index+=1
print(len(forDelete), " lines need to be suppressed to meet all requirements")

6  lines need to be suppressed to meet all requirements


In [28]:
if LOCAL_SUPRESS:
    print("AFTER LOCAL SUPPRESSION")
    df = df.drop(df.index[forDelete])
    new_dict, new_k_anon = get_dict_and_k_anonimity()
    print("New k-anon = ", new_k_anon)
    print("New unique rows:", len(new_dict))
    sorted_new_dict = sorted(new_dict.items(), key=lambda x:x[1])
    print("New 5 least frequent rows:")
    for i in range(5):
        print(sorted_new_dict[i])

AFTER LOCAL SUPPRESSION
New k-anon =  5
New unique rows: 4801
New 5 least frequent rows:
('***Beelineпр-кт Малый В.О. ***ПрограммистVery high', 5)
('***MTSпр-кт Малый В.О. ***ПрограммистVery high', 5)
('***BeelineОбводного канала наб. ***ПрограммистVery high', 5)
('***Tele2ул. Разъезжая ***АвтослесарьMid', 6)
('***Tele2ул. Потёмкинская ***ПрограммистVery high', 6)


In [34]:
sum_of_frequencies = 0
def get_dict():
    if LOCAL_SUPRESS:
        return new_dict
    else:
        return mydict
for key in get_dict():
    sum_of_frequencies += get_dict()[key]
print("AVG K-anon = ", round(sum_of_frequencies / len(get_dict()), 2))

AVG K-anon =  207.79


In [29]:
path = filename.split('.')[0] + 'anonymized.xlsx'

if SAVE_ANON_FILE:
    df.to_excel(path, index = False)

In [ ]:
df